In [162]:
import numpy 
import pandas 
import nltk
import csv
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

##### Los topicos se encuentran juntos en el dataframe, esta función los separa de acuerdo al tema, recibe el dataframe y devuelve una lista con los titulares de un solo tópico

In [28]:
def separaTopico(topico, df): #Separa todos los encabezados de acuerdo a su tópico
    listTopic = []
    dataArr = numpy.asarray(df)
    for i in range(len(dataArr)):
        if(dataArr[i][1] == topico):
            listTopic.append(dataArr[i][0])
    return listTopic

##### Cuando se tienen las palabras unicas de cada tópico, se eliminan caracteres especiales. Retorna una lista con las palabras que componen el texto

In [164]:
def limpiaTexto(text):
    "List all the word tokens in a text."
    return re.findall('[a-zA-Z0-9\-áéíóúÁÉÍÓÚ]{3,254}', text.lower())


##### Esta función busca eliminar palabras plurales para quedarnos únicamente con las singulares a través de la raíz de la palabra 

In [30]:
def RaizSingular(palabra):
    ultima = len(palabra) -1
    raiz = []
    plural = False

    if (palabra[ultima] == 's'): #Verificamos si la última letra es 's'
        if (palabra[ultima - 1] == 'a' or 
            palabra[ultima - 1] == 'i' or
            palabra[ultima - 1] == 'o' or
            palabra[ultima - 1] == 'u'):
            
            for i in range(ultima):
                    raiz.append(palabra[i])
            
            palabra = "".join(raiz)
            return palabra
            
        if(palabra[ultima - 1] == 'e'):
            if (palabra[ultima - 2] == 'c'):
                for i in range(ultima - 2):
                    raiz.append(palabra[i])
                    
                raiz.append('z')
                    
                palabra = "".join(raiz)
                return palabra
            
            elif (palabra[ultima - 2] != 'a' or
                palabra[ultima - 2] != 'e' or
                palabra[ultima - 2] != 'i' or
                palabra[ultima - 2] != 'o' or
                palabra[ultima - 2] != 'u'):
                
                for i in range(ultima - 1):
                    raiz.append(palabra[i])
                    
                palabra = "".join(raiz)
                return palabra
                                                                        
    else:
        return palabra

##### Esta función sustituye las tildes de las vocales por unicamente las vocales

In [31]:
def cleanTildes(palabra):
    s = palabra
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
    )
    
    for a, b in replacements:
        s = s.replace(a, b).replace(a.upper(), b.upper())
    
    return s

In [112]:
def limpiaPalabra(arrPalabras):
    raiz = []
    for i in range(len(arrPalabras)):
        #raiz.append(RaizSingular(cleanTildes(arrPalabras[i])))
        raiz.append(cleanTildes(arrPalabras[i]))
    
    return numpy.asarray(raiz, dtype=str)


In [121]:
aux = "auxílio"
print(limpiaPalabra(aux))

['a' 'u' 'x' 'i' 'l' 'i' 'o']


##### Cada encabezado corresponde a un documento, esta función separa cada palabra para agregarlas a un solo array

In [113]:
def PalabrasTopico(listTopico): #Devuelve todas las palabras que aparecen en los titulares de las noticias 
    listHeadLines = []
    for i in range(len(listTopico)):
        #aux = nltk.word_tokenize(arrayTopico[i])
        aux = limpiaTexto(listTopico[i])
        #aux = limpiaPalabra()
        for j in range(len(aux)):
            aux2 = aux[j]
            listHeadLines.append(aux2)
            
    return numpy.asarray(listHeadLines)

In [34]:
def Probabilidad(palabrasTopico, palabrasUnicasTopico): #Devuelve un array con la ocurrencias de cada única del tópico
    Prob = numpy.zeros(len(palabrasUnicasTopico))
    for i in range(len(palabrasUnicasTopico)):
        Prob[i] = numpy.sum(palabrasTopico == palabrasUnicasTopico[i])
        Prob[i] = Prob[i] /len(palabrasTopico)
        
    return Prob

##### Recibe el vector de probabilidades y elige la máxima para finalmente devolver el tópico al que pertenece

In [46]:
def Clasificador(arrayProbabilidades):
    indice = numpy.where(arrayProbabilidades == max(arrayProbabilidades))[0][0]
    #print("Max = ", max(arrayProbabilidades))
    if (indice == 0):
        #print("Covid")
        return 'covid'
    if (indice == 1):
        #print("Tecnología")
        return 'tecnologia'
    if (indice == 2):
        #print("Deportes")
        return 'deportes'
    if (indice == 3):
        #print("Economía")
        return 'economia'
    if (indice == 4):
        #print("Cultura")
        return 'cultura'
    return 'otros'

##### Devuelve la probabilidad máxima

In [47]:
def MAP(tweet, palabrasCovid, AlfabetoCovid, palabrasTecn, AlfabetoTecn, palabrasDep, AlfabetoDep):
    T = nltk.word_tokenize(tweet)
    Proba = numpy.ones(5)

    #Covid
    for j in range(len(T)):
        aux = numpy.sum(palabrasCovid == T[j]) + 1
        Proba[0] *= (aux + 1) / (len(AlfabetoCovid) + aux)
    
    #Tecnologia
    for j in range(len(T)):
        aux = numpy.sum(palabrasTecn == T[j]) 
        Proba[1] *= (aux + 1) / (len(AlfabetoTecn) + aux)

    #Deportes
    for j in range(len(T)):
        aux = numpy.sum(palabrasDep == T[j]) + 1
        Proba[2] *= (aux + 1) / (len(AlfabetoDep) + aux)
        
    #Economía
    for j in range(len(T)):
        aux = numpy.sum(palabrasEcon == T[j]) + 1
        Proba[3] *= (aux + 1) / (len(AlfabetoEcon) + aux)
        
    #Economía
    for j in range(len(T)):
        aux = numpy.sum(palabrasCult == T[j]) + 1
        Proba[4] *= (aux + 1) / (len(AlfabetoCult) + aux)

    for p in range(len(Proba)):
        Proba[p] = Proba[p] * 1/5
        
    return Clasificador(Proba)
        

#### Abrimos el dataset con pandas
Se abre el dataset de titulares y se realiza un preprocesamiento el cual consiste en eliminar tildes y eliminar palabras que tengan una longitud menor a tres 

In [108]:
nombres = ['Titular', 'Tópico']
dataTrain = pandas.read_csv('.//TitularesTrain//titulares.csv', names=nombres)
dataTrain = dataTrain.dropna()
dataTrain

,Titular,Tópico
0,Cierran rastro de Mérida tras detectar 17 cont...,covid
2,El “peor escenario” para Tabasco eran 200 muer...,covid
4,Jalisco instalará espacios para enfermos no gr...,covid
6,Guerrero no está listo para iniciar la “nueva ...,covid
8,Reportan la muerte de 364 personas por covid-1...,covid
...,...,...
7846,"“El Cascanueces” de Disney: Cuento de hadas, m...",cultura
7848,"Rock, blues y música alternativa en Sinaloa",cultura
7850,María Mercedes Coroy es “Malinche” en Canal Once,cultura
7852,“Motivos para el canto y la danza. Poesía del 68”,cultura


##### Separamos del dataset por tópicos y a cada encabezado lo separamos por palabra de los documentos eliminando caracteres especiales

In [125]:
palabrasCovid = limpiaPalabra(PalabrasTopico(separaTopico('covid', dataTrain)))
AlfabetoCovid = numpy.unique(palabrasCovid)
print(len(palabrasCovid))


palabrasTecn = limpiaPalabra(PalabrasTopico(separaTopico('tecnologia', dataTrain)))
AlfabetoTecn = numpy.unique(palabrasTecn)
#print(len(palabrasTecn))
#print(AlfabetoTecn)

palabrasDep = limpiaPalabra(PalabrasTopico(separaTopico('deportes', dataTrain)))
AlfabetoDep = numpy.unique(palabrasDep)

palabrasEcon = limpiaPalabra(PalabrasTopico(separaTopico('economia', dataTrain)))
AlfabetoEcon = numpy.unique(palabrasEcon)

palabrasCult = limpiaPalabra(PalabrasTopico(separaTopico('cultura', dataTrain)))
AlfabetoCult = numpy.unique(palabrasCult)

Alfabeto = numpy.concatenate((AlfabetoCovid, AlfabetoTecn, AlfabetoDep, AlfabetoEcon, AlfabetoCult))
#Alfabeto.shape

9199


##### Obtenemos las probabilidades a priori, haciendo cada una de estas equiprobabloe $ P(C) $

In [126]:
PCovid = 1/5
PTecn = 1/5
PDep = 1/5
PEcon = 1/5
PCult = 1/5
print("PCovid = ", PCovid, "PTecn = ", PTecn, "PDep = ", PDep, "PEcon = ", PEcon, "PCult = ", PCult)

PCovid =  0.2 PTecn =  0.2 PDep =  0.2 PEcon =  0.2 PCult =  0.2


##### Obtenemos la frecuencia de palabras de cada atributo (palabra) $ P(A_{i} | C_{j}) $

In [127]:
ProbaCov = Probabilidad(palabrasCovid, AlfabetoCovid)
print(ProbaCov)
ProbaTecn = Probabilidad(palabrasTecn, AlfabetoTecn)
ProbaDep = Probabilidad(palabrasDep, AlfabetoDep)
ProbaEcon = Probabilidad(palabrasEcon, AlfabetoEcon)
ProbaCult = Probabilidad(palabrasCult, AlfabetoCult)

[0.00021741 0.00054354 0.00010871 ... 0.00054354 0.00010871 0.00010871]


##### TEST

In [128]:
dataTest = pandas.read_csv('.//TitularesTrain//Test.csv', names=nombres)
dataTest = dataTest.dropna()
dataTest

,Titular,Tópico
0,Inicia esta semana la entrega de créditos a mi...,covid
2,El IPN retomará clases el 1 de junio y se exte...,covid
4,Coparmex urge a trabajadores a que exijan el s...,covid
6,AMLO prevé viajar a EU para agradecer a Trump ...,covid
8,La pandemia saca a flote la desigualdad en acc...,covid
...,...,...
418,Una breve historia fílmica de Venom,cultura
420,"En la cocina de Pati Jinich, chef mexicana rec...",cultura
422,Actividades por el homenaje a Arreola y estren...,cultura
424,"“Balam Antsetik. La Segunda Era”, de Margarita...",cultura


In [165]:
def preProcesamiento(listaTitulares):
    titulares = []
    for titular in listaTitulares:
        aux = limpiaTexto(titular)
        titulares.append(str.join(' ', aux))
    
    return titulares


In [168]:
listTest = dataTest.Titular.values.tolist()
arrTest = numpy.asarray(preProcesamiento(listTest))

covTrue = 0
TecnTrue = 0
DepTrue = 0
EconTrue = 0
CultTrue = 0
for i in range(len(arrTest)):
    if (arrTestTopic[i] == 'covid'):
        covTrue += 1
    if (arrTestTopic[i] == 'tecnologia'):
        TecnTrue += 1
    if (arrTestTopic[i] == 'deportes'):
        DepTrue += 1
    if (arrTestTopic[i] == 'economia'):
        EconTrue += 1
    if (arrTestTopic[i] == 'cultura'):
        CultTrue += 1
        
print("Covid = ", covTrue, "Tecnología = ", TecnTrue, "Deportes = ", DepTrue, "Economia = ", EconTrue, "Cultura= ", CultTrue)

Covid =  35 Tecnología =  25 Deportes =  40 Economia =  65 Cultura=  49


In [169]:
result = []
for i in range(len(arrTest)):
    result.append(MAP(arrTest[i], palabrasCovid, AlfabetoCovid, palabrasTecn, AlfabetoTecn, palabrasDep, AlfabetoDep))
    
resultClass = numpy.asarray(result)


In [198]:
resultCov = 0
for i in range(len(resultClass)):
    #print(resultClass[i], arrTestTopic[i])
    if (arrTestTopic[i] == 'cultura' and resultClass[i] == 'cultura'):
        resultCov += 1
        
print(resultCov)

24
